In [19]:
%matplotlib inline
import pandas as pd
import xgboost as xgb

In [20]:
from sklearn.datasets import load_iris

iris = load_iris()

iris_data = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_target = pd.Series(iris.target)

In [21]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(iris_data, iris_target, test_size=0.8, shuffle=True)

In [22]:
params = {'eta': [0.01, 0.1, 1.0], 'gamma': [0], 
                  'n_estimators': [100], 'max_depth':[2], 'min_child_weigh': [1], 
                  'nthread': [2] }

In [23]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV

model = xgb.XGBClassifier()
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
clf = GridSearchCV(estimator=model, param_grid=params, cv=skf, scoring="accuracy", n_jobs=1, verbose=3)
clf.fit(train_x, train_y)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=1.000, total=   0.0s
[CV] eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=0.833, total=   0.0s
[CV] eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=1.000, total=   0.0s
[CV] eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=1.000, total=   0.0s
[CV] eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_esti

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=1.000, total=   0.0s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=1.000, total=   0.0s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=1.000, total=   0.0s
[CV] eta=1.0, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=1.0, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=1.000, total=   0.0s
[CV] eta=1.0, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=1.0, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=0.833, total=   0.0s
[CV] eta=1.0, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=1.0, gamma

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.4s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_wei...
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                     validate_parameters=False,
                                     verbosity=None),
       

In [24]:
print(clf.cv_results_['mean_test_score'])

[0.96666667 0.96666667 0.96666667]


In [25]:
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
params = clf.cv_results_['params']

df = pd.DataFrame(data=zip(means, stds, params), columns=['mean', 'std', 'params'])
df = df.sort_values('std', ascending=True)
df = df.sort_values('mean', ascending=False)

for index, row in df.iterrows():
    print("score: %.3f +/-%.4f, params: %r" % (row['mean'], row['std']*2, row['params']))

score: 0.967 +/-0.1333, params: {'eta': 0.01, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 100, 'nthread': 2}
score: 0.967 +/-0.1333, params: {'eta': 0.1, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 100, 'nthread': 2}
score: 0.967 +/-0.1333, params: {'eta': 1.0, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 100, 'nthread': 2}


In [26]:
print("Best score: %.4f" % (clf.best_score_))
print(clf.best_params_)

Best score: 0.9667
{'eta': 0.01, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 100, 'nthread': 2}


In [27]:
from sklearn.metrics import accuracy_score

model = clf.best_estimator_
pred = model.predict(test_x)
score = accuracy_score(test_y, pred)
print('score:{0:.4f}'.format(score))

score:0.9417


In [28]:
params = {'eta': [0.1], 'gamma': [0], 
                  'n_estimators': [50, 100, 200], 'max_depth':[2], 'min_child_weigh': [1], 
                  'nthread': [2] }

In [29]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV

model = xgb.XGBClassifier()
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
clf = GridSearchCV(estimator=model, param_grid=params, cv=skf, scoring="accuracy", n_jobs=1, verbose=3)
clf.fit(train_x, train_y)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2, score=1.000, total=   0.0s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2, score=0.833, total=   0.0s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2, score=1.000, total=   0.0s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2, score=1.000, total=   0.0s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=1.000, total=   0.1s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=1.000, total=   0.0s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=1.000, total=   0.0s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=200, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=200, nthread=2, score=1.000, total=   0.1s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=200, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=200, nthread=2, score=0.833, total=   0.1s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=200, nthread=2 
[CV]  eta=0.1, gamma

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.6s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_wei...
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                     validate_parameters=False,
                                     verbosity=None),
       

In [30]:
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
params = clf.cv_results_['params']

df = pd.DataFrame(data=zip(means, stds, params), columns=['mean', 'std', 'params'])
df = df.sort_values('std', ascending=True)
df = df.sort_values('mean', ascending=False)

for index, row in df.iterrows():
    print("score: %.3f +/-%.4f, params: %r" % (row['mean'], row['std']*2, row['params']))

score: 0.967 +/-0.1333, params: {'eta': 0.1, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 50, 'nthread': 2}
score: 0.967 +/-0.1333, params: {'eta': 0.1, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 100, 'nthread': 2}
score: 0.967 +/-0.1333, params: {'eta': 0.1, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 200, 'nthread': 2}


In [31]:
print("Best score: %.4f" % (clf.best_score_))
print(clf.best_params_)

Best score: 0.9667
{'eta': 0.1, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 50, 'nthread': 2}


In [32]:
from sklearn.metrics import accuracy_score

model = clf.best_estimator_
pred = model.predict(test_x)
score = accuracy_score(test_y, pred)
print('score:{0:.4f}'.format(score))

score:0.9417


In [33]:
params = {'eta': [0.1], 'gamma': [0, 0.1, 0.2], 
                  'n_estimators': [50], 'max_depth':[2, 4], 'min_child_weigh': [1, 2], 
                  'nthread': [2] }

In [34]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV

model = xgb.XGBClassifier()
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
clf = GridSearchCV(estimator=model, param_grid=params, cv=skf, scoring="accuracy", n_jobs=1, verbose=3)
clf.fit(train_x, train_y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2, score=1.000, total=   0.0s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2, score=0.833, total=   0.0s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2, score=1.000, total=   0.0s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2, score=1.000, total=   0.0s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  eta=0.1, gamma=0, max_depth=4, min_child_weigh=1, n_estimators=50, nthread=2, score=1.000, total=   0.0s
[CV] eta=0.1, gamma=0, max_depth=4, min_child_weigh=2, n_estimators=50, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=4, min_child_weigh=2, n_estimators=50, nthread=2, score=1.000, total=   0.0s
[CV] eta=0.1, gamma=0, max_depth=4, min_child_weigh=2, n_estimators=50, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=4, min_child_weigh=2, n_estimators=50, nthread=2, score=0.833, total=   0.0s
[CV] eta=0.1, gamma=0, max_depth=4, min_child_weigh=2, n_estimators=50, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=4, min_child_weigh=2, n_estimators=50, nthread=2, score=1.000, total=   0.0s
[CV] eta=0.1, gamma=0, max_depth=4, min_child_weigh=2, n_estimators=50, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=4, min_child_weigh=2, n_estimators=50, nthread=2, score=1.000, total=   0.0s
[CV] eta=0.1, gamma=0, max_depth=4, min_child_weigh=2, n_estimators=50, nthread=2 
[CV]  eta=0.1, gamma=0, max_de

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.8s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_wei...
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                     validate_parameters=False,
                                     verbosity=None),
       

In [35]:
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
params = clf.cv_results_['params']

df = pd.DataFrame(data=zip(means, stds, params), columns=['mean', 'std', 'params'])
df = df.sort_values('std', ascending=True)
df = df.sort_values('mean', ascending=False)

for index, row in df.iterrows():
    print("score: %.3f +/-%.4f, params: %r" % (row['mean'], row['std']*2, row['params']))

score: 0.967 +/-0.1333, params: {'eta': 0.1, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 50, 'nthread': 2}
score: 0.967 +/-0.1333, params: {'eta': 0.1, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 2, 'n_estimators': 50, 'nthread': 2}
score: 0.967 +/-0.1333, params: {'eta': 0.1, 'gamma': 0, 'max_depth': 4, 'min_child_weigh': 1, 'n_estimators': 50, 'nthread': 2}
score: 0.967 +/-0.1333, params: {'eta': 0.1, 'gamma': 0, 'max_depth': 4, 'min_child_weigh': 2, 'n_estimators': 50, 'nthread': 2}
score: 0.967 +/-0.1333, params: {'eta': 0.1, 'gamma': 0.1, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 50, 'nthread': 2}
score: 0.967 +/-0.1333, params: {'eta': 0.1, 'gamma': 0.1, 'max_depth': 2, 'min_child_weigh': 2, 'n_estimators': 50, 'nthread': 2}
score: 0.967 +/-0.1333, params: {'eta': 0.1, 'gamma': 0.1, 'max_depth': 4, 'min_child_weigh': 1, 'n_estimators': 50, 'nthread': 2}
score: 0.967 +/-0.1333, params: {'eta': 0.1, 'gamma': 0.1, 'max_depth': 4, 'min_child_weigh

In [36]:
print("Best score: %.4f" % (clf.best_score_))
print(clf.best_params_)

Best score: 0.9667
{'eta': 0.1, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 50, 'nthread': 2}


In [37]:
from sklearn.metrics import accuracy_score

model = clf.best_estimator_
pred = model.predict(test_x)
score = accuracy_score(test_y, pred)
print('score:{0:.4f}'.format(score))

score:0.9417
